#### What We will Learn

1. S3 Buckets- Boto3
2. Iam Roles and Users
3. Complete Infrastructure of AWS Sagemaker-Training, Endpoints

In [19]:
import sagemaker 
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd 

sm_boto3 = boto3.client("sagemaker") #this text in the inverted "" is fixed not custom made and there are diff kinds for diff work
sess = sagemaker.Session()
region = sess.boto_session.region_name

bucket = "mobbucketsagemaker555" # this is the name of the s3 bucket that we want to create 
#we need to go an create the s3 bucket in AWS wih the same name, if not available , paste here the name with which u have created 
print("Using bucket" + bucket)

Using bucketmobbucketsagemaker555


In [3]:
region

'eu-north-1'

In [4]:
df = pd.read_csv("mob_price_classification_train.csv")  
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
df.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [6]:
df['price_range'].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [11]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [12]:
label = features.pop(-1)
label

'price_range'

In [13]:
x= df[features]
y= df[label]

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1600, 20)
(400, 20)
(1600,)
(400,)


In [17]:
#here we are doing this because we can save them in the s3 bucket or local for future use 
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
968,1923,0,0.5,1,7,0,46,0.5,191,1,...,767,1759,1489,10,9,3,1,1,1,1
240,633,1,2.2,0,0,1,49,0.1,139,8,...,529,1009,3560,11,1,16,1,1,1,2
819,1236,0,0.9,1,2,1,57,0.1,188,1,...,517,809,1406,14,12,20,1,0,1,0
692,781,0,1.1,0,2,0,38,0.4,198,5,...,304,1674,3508,13,8,5,0,0,1,3
420,1456,1,0.5,1,7,0,7,0.4,105,5,...,823,1104,1587,6,5,20,1,0,1,1


In [18]:
trainX.to_csv("train-V-1.csv", index = False)
testX.to_csv("test-V-1.csv", index=False)

In [20]:
bucket

'mobbucketsagemaker555'

In [ ]:
##send the data to S3, sagemaker will take the data for training fron S3

#here we need to first mention the folder inside the bucket where we will upload the data
#Here whatever folders and subfolders that we mention will be automatically created in the bucket with the below code
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"
trainpath = sess.upload_data(path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix)
testpath = sess.upload_data(path = "test-V-1.csv", bucket = bucket, key_prefix=sk_prefix)

print(trainpath)
print(testpath)

s3://mobbucketsagemaker555/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
s3://mobbucketsagemaker555/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


### Script that will be used by AWS sagemaker to train the models


In [68]:
%%writefile script.py 
#This will automatically create this py file and write all the code in this cell in that file

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
import sklearn
import joblib 
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ == "__main__":
    print("[Info] Extracting Arguments")

    parser = argparse.ArgumentParser()

    ##Hyper parameter
    parser.add_argument("--n_estimators", type= int, default=100)
    parser.add_argument("--random_state", type= int, default=42)

    ###Data . model and output directories 
    # ( These env variables are already created by the sage maker after training with the given names only)
    parser.add_argument("--model-dir", type=str, default = os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type= str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default = os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args,_ =parser.parse_known_args()

    print("Sklearn version", sklearn.__version__)
    print("Joblib version", joblib.__version__)

    print("[INFO] Reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building train and test data")
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features, "\n")

    print("Label column is: ", label)
    print()


    print("Data Shape: ")
    print()

    print("--- SHAPE OF TRAINING DATA (80%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()

    print("Training Random Forest Model")
    model=RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state,
                                 verbose=2, n_jobs=1)
    
    model.fit(X_train, y_train)

    print()

    model_path = os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model, model_path)

    print("Model saved at :",  model_path)

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)

    print()

    print("---METRICS RESULTS FOR TESTING DATA----")
    print()
    print("Total rows are : ", X_test.shape[0])
    print('[TESTING] testing report: ')
    print(test_rep)

Overwriting script.py


### AWS sagemaker entry point to execute the training script

In [48]:
from sagemaker import image_uris
image_uris.retrieve(framework='sklearn', region='eu-north-1', version='1.0-1', py_version='py3')


INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


'662702820516.dkr.ecr.eu-north-1.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3'

In [49]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION="1.0-1"

sklearn_estimators = SKLearn(
    # py_version = "py3",
    entry_point="script.py",
    role= "arn:aws:iam::182896147894:role/sagemakeraccess",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name = "RF-custom-sklearn",
    hyperparameters={
        "n_estimators":100,
        "random_state":0
    },
    use_spot_instance = True,
    max_run = 3600 
)

In [50]:
# launch training  job, with asynchronous call
sklearn_estimators.fit({"train": trainpath, "test": testpath}, wait= True)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2025-06-28-18-54-47-375


2025-06-28 18:54:57 Starting - Starting the training job...
2025-06-28 18:55:31 Starting - Preparing the instances for training...
2025-06-28 18:55:50 Downloading - Downloading input data...
2025-06-28 18:56:15 Downloading - Downloading the training image...
2025-06-28 18:57:16 Training - Training image download completed. Training in progress..2025-06-28 18:57:19,556 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-06-28 18:57:19,560 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-28 18:57:19,563 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-06-28 18:57:19,579 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-06-28 18:57:19,839 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-28 18:57:19,842 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
20

### To get the model from S3

In [51]:
sklearn_estimators.latest_training_job.wait(logs=None)
artifact= sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimators.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

2025-06-28 18:57:40 Starting - Preparing the instances for training
2025-06-28 18:57:40 Downloading - Downloading the training image
2025-06-28 18:57:40 Training - Training image download completed. Training in progress.
2025-06-28 18:57:40 Uploading - Uploading generated training model
2025-06-28 18:57:40 Completed - Training job completed2025-06-28 18:57:19,556 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-06-28 18:57:19,560 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-28 18:57:19,563 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-06-28 18:57:19,579 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-06-28 18:57:19,839 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-28 18:57:19,842 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-06

In [52]:
artifact

's3://sagemaker-eu-north-1-182896147894/RF-custom-sklearn-2025-06-28-18-54-47-375/output/model.tar.gz'

### Deploy the model for endpoints

In [70]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name="Custom-sklearn-model-"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role = "arn:aws:iam::182896147894:role/sagemakeraccess",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION
)

In [71]:
model

In [72]:
## Endpoint deployment

endpoint_name= "Custom-sklearn-model-"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(f"End-point name = {endpoint_name}" )

predictor=model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name=endpoint_name
)                        

End-point name = Custom-sklearn-model-2025-06-28-19-40-25


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2025-06-28-19-40-16
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2025-06-28-19-40-25
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2025-06-28-19-40-25


------!

In [73]:
predictor

In [74]:
testX[features][0:2]

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
1860,1646,0,2.5,0,3,1,25,0.6,200,2,5,211,1608,686,8,6,11,1,1,0
353,1182,0,0.5,0,7,1,8,0.5,138,8,16,275,986,2563,19,17,19,1,0,0


In [76]:
from sagemaker.serializers import CSVSerializer

predictor.serializer = CSVSerializer()
# predictor.deserializer = CSVDeserializer()

print(predictor.predict(testX[features][:5]))

[0 2 1 3 1]


In [77]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '4b2874c1-254c-42a5-851b-b3640bafafae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4b2874c1-254c-42a5-851b-b3640bafafae',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 28 Jun 2025 19:45:54 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}